In [61]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By

In [72]:
url = "https://inmu2.mahidol.ac.th/thaifcd/search_food_by_group.php"
driver_path = "../driver/chromedriver.exe"
export_path = "../th_food_item_options_raw.csv"

In [73]:
serv = Service(driver_path)
# opt = webdriver.ChromeOptions()
# opt.add_argument("--log-level 3")
driver = webdriver.Chrome(service=serv)  # , options=opt)
driver.get(url)

food_groups_sb = driver.find_element(By.XPATH, '//*[@id="frmfood_group_id"]')
main_search_btn = driver.find_element(
    By.XPATH, '//*[@id="form1"]/div[1]/table/tbody/tr[3]/td[2]/input'
)

food_groups = [x.text for x in food_groups_sb.find_elements(By.TAG_NAME, "option")]


In [74]:
food_group = food_groups[-1]

In [75]:
food_groups_sb.send_keys(food_group)
main_search_btn.click()
food_tb = driver.find_element(By.XPATH, '//*[@id="form1"]/div[4]/table')

In [76]:
row = food_tb.find_elements(By.TAG_NAME, "tr")[1]

In [77]:
row_data = []
nutr_link = ""
for cell in row.find_elements(By.TAG_NAME, "td")[1:]:
    if cell.text is None:
        continue

    row_data.append(cell.text)
    if nutr_link == "" and cell.find_elements(By.CSS_SELECTOR, "a"):
        nutr_link = cell.find_element(By.TAG_NAME, "a").get_attribute("href")

In [78]:
row_data

['มอลคิสท์ แครกเกอร์, มอลคิสท์',
 'Cracker',
 'พีที มาโยรา อินดาซ์ ทีบีเค ประเทศอินโดนีเซีย',
 'Z060001']

In [79]:
food_info_output = []
col_names = ['thai_name', 'english_name', 'scientific_name']
food_id = row_data[-1]
for k, v in zip(col_names, row_data):
    food_info_output.append([food_id, k, v, ''])

In [80]:
food_info_output

[['Z060001', 'thai_name', 'มอลคิสท์ แครกเกอร์, มอลคิสท์', ''],
 ['Z060001', 'english_name', 'Cracker', ''],
 ['Z060001',
  'scientific_name',
  'พีที มาโยรา อินดาซ์ ทีบีเค ประเทศอินโดนีเซีย',
  '']]

In [81]:
nutr_link

'https://inmu2.mahidol.ac.th/thaifcd/search_food_by_name_result.php?food_id=1936&rk='

In [82]:
driver.execute_script("window.open('');")
driver.switch_to.window(driver.window_handles[1])
driver.get(nutr_link)

nutr_baseweight = driver.find_element(By.XPATH, '//*[@id="form1"]/div[4]/div/div[1]/div/div[3]')
food_info_output.append([food_id, 'base_weight', nutr_baseweight.text, ''])


In [86]:
nutr_tb = driver.find_element(By.XPATH, '//*[@id="div_table_body"]')
for row in nutr_tb.find_elements(By.TAG_NAME, "div"):
    cells = [cell.text for cell in row.find_elements(By.TAG_NAME, "div")]
    if len(cells) != 4:
        continue
    k, unit, _, v = cells
    food_info_output.append([food_id, k, v, unit])

In [87]:
food_info_output

[['Z060001', 'thai_name', 'มอลคิสท์ แครกเกอร์, มอลคิสท์', ''],
 ['Z060001', 'english_name', 'Cracker', ''],
 ['Z060001',
  'scientific_name',
  'พีที มาโยรา อินดาซ์ ทีบีเค ประเทศอินโดนีเซีย',
  ''],
 ['Z060001', 'base_weight', 'Serving', ''],
 ['Z060001', 'Total energy - by calculation', '459', 'kcal'],
 ['Z060001', 'Protein, total', '7.41', 'g'],
 ['Z060001', 'Fat, total', '14.81', 'g'],
 ['Z060001', 'Carbohydrate, total', '74.07', 'g'],
 ['Z060001', 'Dietary fibre', '2.6', 'g'],
 ['Z060001', 'Sodium', '593', 'mg'],
 ['Z060001', 'Iron', '3.33', 'mg'],
 ['Z060001', 'Thiamin', '1.94', 'mg'],
 ['Z060001', 'Riboflavin', '1.26', 'mg'],
 ['Z060001', 'Sugars, total', '14.81', 'g'],
 ['Z060001', 'Fatty acids, total saturated', '7.41', 'g']]

In [46]:
with open(export_path, "a", encoding="utf-8") as f:
    for row in food_info_output:
        f.write("|".join(row) + '\n')

In [13]:
from io import StringIO
import lxml.etree
html = driver.execute_script("return document.documentElement.outerHTML")
parser = lxml.etree.HTMLParser()
tree = lxml.etree.parse(StringIO(html), parser)
lxml.etree.tostring(tree)


In [ ]:
for food_group in food_groups[1:]:

    food_groups_sb.send_keys(food_group)
    main_search_btn.click()
    food_tb = driver.find_element(By.XPATH, '//*[@id="form1"]/div[4]/table')

    for row in food_tb.find_elements(By.TAG_NAME, "tr")[1:]:
        # cols = ["thai_name", "eng_name", "sci_name", "code"]
        row_data = []
        nutr_link = ""
        for cell in row.find_elements(By.TAG_NAME, "td")[1:]:
            if cell.text is None:
                continue

            row_data.append(cell.text)
            if nutr_link == "" and cell.find_elements(By.CSS_SELECTOR, "a"):
                nutr_link = cell.find_element(By.TAG_NAME, "a").get_attribute("href")

        # per 100 g
        # row_data.append(nutr_link)
        driver.execute_script("window.open('');")
        driver.switch_to.window(driver.window_handles[1])
        driver.get(nutr_link)
        nutr_tb = driver.find_element(By.XPATH, '//*[@id="div_table_body"]')
        for row in nutr_tb.find_elements(By.TAG_NAME, "div"):
            for cell in row.find_elements(By.TAG_NAME, "div"):
                # print(cell.get_attribute("class"))
                row_data.append(cell.text)

        print(row_data)
        with open(export_path, "a", encoding="utf-8") as f:
            f.write("|".join(row_data) + '\n')
        break
    # back_btn = driver.find_element(By.XPATH, "//*[@id="form1"]/div[5]/a[3]")
    # back_btn.click()

    break


In [71]:
driver.quit()